In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycalphad import Database, calculate,equilibrium, variables as v
from pycalphad.core.calculate import instantiate_models
from pycalphad.plot.utils import phase_legend
import numpy as np

In [3]:
dbf = Database('mcmc_UQ1.tdb')

In [5]:
phases = list(dbf.phases.keys())
phases = ['MU_PHASE']
print(phases)

['MU_PHASE']


In [7]:
model=instantiate_models(dbf, ['NB', 'NI'], ['MU_PHASE'])

In [8]:
a=model['MU_PHASE'].site_fractions

In [9]:
a

[MU_PHASE0NB,
 MU_PHASE0NI,
 MU_PHASE1NB,
 MU_PHASE1NI,
 MU_PHASE2NB,
 MU_PHASE2NI,
 MU_PHASE3NB,
 MU_PHASE3NI,
 MU_PHASE4NB,
 MU_PHASE4NI]

In [6]:
eq = equilibrium(dbf,  ['NB', 'NI'], phases, {v.P: 101325, v.T: 1273, v.X('NB'): 0.569})
result=eq.Y.where(eq.Phase=='MU_PHASE')


<xarray.DataArray 'Y' (N: 1, P: 1, T: 1, X_NB: 1, vertex: 3, internal_dof: 10)>
array([[[[[[2.70453130e-08, 9.99999973e-01, 2.35473654e-07,
            9.99999765e-01, 1.00000000e+00, 9.28973433e-11,
            9.32054190e-01, 6.79458098e-02, 9.32054219e-01,
            6.79457814e-02],
           [7.19182941e-08, 9.99999928e-01, 9.99910222e-01,
            8.97776911e-05, 9.99472810e-01, 5.27190049e-04,
            3.42833037e-03, 9.96571670e-01, 1.84438830e-05,
            9.99981556e-01],
           [           nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan]]]]]])
Coordinates:
  * N        (N) float64 1.0
  * P        (P) float64 1.013e+05
  * T        (T) float64 1.273e+03
  * X_NB     (X_NB) float64 0.569
  * vertex   (vertex) int64 0 1 2
Dimensions without coordinates: internal_dof

In [9]:
np.savetxt('0.569.txt',result[0])
print(result)

<xarray.DataArray 'Y' (N: 1, P: 1, T: 1, X_NB: 1, vertex: 3, internal_dof: 10)>
array([[[[[[8.50321265e-01, 1.49678735e-01, 9.83660615e-01,
            1.63393852e-02, 9.99993368e-01, 6.63169551e-06,
            1.55054147e-01, 8.44945853e-01, 7.98724623e-01,
            2.01275377e-01],
           [           nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan],
           [           nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan,            nan,            nan,
                       nan]]]]]])
Coordinates:
  * N        (N) float64 1.0
  * P        (P) float64 1.013e+05
  * T        (T) float64 1.273e+03
  * X_NB     (X_NB) float64 0.569
  * vertex   (vertex) int64 0 1 2
Dimensions without coordinates: internal_dof


In [9]:
from collections import defaultdict 
fin_re={}
for i in range(len(eq.X_PD)):
    pl_re=[]
    re=result[i].values
    print(re)
    re1 = re[np.logical_not(np.isnan(re))]
    if len(re1)>len(a):
        re1 = re1[range(0,len(a))]
    print('re1',re1)
    pl_re.append(re1)
    fin_re[str(i)]=pl_re

AttributeError: 'Dataset' object has no attribute 'X_PD'

In [ ]:
value_re=[]
num=len(a)/2
a_value=[]
for j in range(int(num)):
    value=[]
    a_value.append(a[j*2])
    for key in fin_re.keys():
        if len(fin_re[key][0])>0:
            value.append(fin_re[key][0][j*2])
        else:
            value.append(0)
    if len(value_re)>0:
        value_re=np.vstack((value_re,value))
    else:
        value_re=value
plt.plot(eq.X_PD.values,np.transpose(value_re))
plt.xlabel('X(PD)',fontsize=15)
plt.ylabel('Site_fraction',fontsize=16)
plt.legend(a_value)
plt.xlim(0,1)
plt.ylim(0,1.1)